# Kunskapskontroll

### Fyll i uppgifterna nedan innan du lämnar in på LearnPoint: 
Namn på samtliga gruppmedlemmar: 

Asami, Guy, Mustafa,  Pawel, Vidar

# Code

### Förbered data

Konvertera ocean_proximity med ordinal encoding (1-4), ta bort ISLAND

Ta bort > 500,000 från kolumnen median_house_value

Ta bort > 51 från kolumnen median_house_age

Fyll tomma värden med medelvärde/median i kolumnen total_bedrooms

Dela datan i train- och test-set

Ta bort/kombinera viss kolumner till nya värden?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder


In [ ]:
housing = pd.read_csv('data/housing.csv')


In [ ]:
# Filter the data and assign it to a new DataFrame
filtered_housing = housing[(housing['median_house_value'] <= 500000) & (housing['housing_median_age'] <= 51) & (housing['ocean_proximity'] != 'ISLAND')].copy()


In [ ]:
category_mapping = {'INLAND': 0, '<1H OCEAN': 1, 'NEAR OCEAN': 2, 'NEAR BAY': 3}
categories = list(category_mapping.keys())
encoder = OrdinalEncoder(categories=[categories])
filtered_housing['ocean_proximity_encoded'] = encoder.fit_transform(filtered_housing[['ocean_proximity']])


In [ ]:
# Calculate the mean of the 'total_bedrooms' column in filtered_housing
mean_total_bedrooms = filtered_housing['total_bedrooms'].mean()

# Fill the missing values with the mean in filtered_housing
filtered_housing.loc[:, 'total_bedrooms'] = filtered_housing['total_bedrooms'].fillna(mean_total_bedrooms)


In [ ]:
print("Column Names:")
print(filtered_housing.columns)
print("\n---\n")

print("First 5 Rows:")
print(filtered_housing.head())
print("\n---\n")

print("DataFrame Info:")
print(filtered_housing.info())
print("\n---\n")

print("Statistical Summary:")
print(filtered_housing.describe())
print("\n---\n")

print("Number of Unique Values in Each Column:")
print(filtered_housing.nunique())
print("\n---\n")

print("Number of Rows in the DataFrame:")
print(len(filtered_housing))


In [ ]:
X = housing.drop(['median_house_value'], axis=1)
y = housing['median_house_value']

## EDA